importing libraries

In [28]:
import json as js
from collections import defaultdict
import nltk
from tqdm import tqdm
import numpy as np
from gensim.models import Word2Vec
import tensorflow as tf
from keras import layers, optimizers
from keras.models import Model, Sequential #type: ignore
from keras.layers import Input, LSTM, Dense, TimeDistributed, Embedding #type: ignore
from keras.callbacks import ModelCheckpoint #type: ignore

loading paths

In [2]:
with open('Config_RNN.json','r') as file:
    paths = js.load(file)

loading padded captions

In [3]:
with open(paths["Padded_preprocessed_data"],'r') as f:
    padded_captions = js.load(f)

In [4]:
padded_captions[0]['input']

'man helmet moped dirt road                                                                                                                                    '

In [24]:
inp=[]
output=[]
max_inp_length=0
max_out_length=0
for i in padded_captions:
    inp.append(i['input'])
    output.append(i['output'])
    max_inp_length=max(max_inp_length,len(i['input'].split()))
    max_out_length=max(max_out_length,len(i['output'].split()))
    

In [ ]:
max_vocab_size = 10000  



input_vectorizer = layers.TextVectorization(
    max_tokens=max_vocab_size,
    output_mode='int',
    output_sequence_length=max_inp_length
)
input_vectorizer.adapt(inp)
input_padded = input_vectorizer(inp)

# TextVectorization for output (captions)
output_vectorizer = layers.TextVectorization(
    max_tokens=max_vocab_size,
    output_mode='int',
    output_sequence_length=max_out_length
)
output_vectorizer.adapt(output)
output_vectorized = output_vectorizer(output)

# Prepare decoder input and output
decoder_input = output_vectorized[:, :-1]  # all except last word
decoder_output = output_vectorized[:, 1:]
vocab_inp = input_vectorizer.get_vocabulary()
vocab_inp_size = len(vocab_inp)+1
vocab_out = output_vectorizer.get_vocabulary()
vocab_out_size = len(vocab_out)+1


Build vocabulary and one hot encoding

In [26]:
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(vocab_inp_size, 128)(encoder_inputs)
encoder_lstm, state_h, state_c = LSTM(158, return_state=True)(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(vocab_out_size, 128)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(158, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(vocab_out_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, None, 128) │  1,280,000 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, None, 128) │  1,280,000 │ input_layer_3[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ [(None, 158),     │    181,384 │ embedding_3[0][0] │
│                     │ (None, 158),      │            │                   │
│                     │ (None, 158)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ [(None, None,     │    181,384 │ embedding_4[0][0… │
│                     │ 158), (None,      │            │ lstm_4[0][1],     │
│                     │ 158), (None,      │            │ lstm_4[0][2]      │
│                     │ 158)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, None,      │  1,590,000 │ lstm_5[0][0]      │
│                     │ 10000)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,512,768 (17.21 MB)

 Trainable params: 4,512,768 (17.21 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
checkpoint_cb = ModelCheckpoint(
    filepath=paths['Trained_model_RNN'],       # where to save the best model
    monitor='loss',                 # monitor training loss (or use 'val_loss' if you have validation)
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

In [ ]:
model.fit(
    [input_padded, decoder_input],
    np.expand_dims(decoder_output, -1),
    batch_size=32,
    epochs=20,
    callbacks=[checkpoint_cb]
)

Epoch 1/20
18492/18493 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.6514
Epoch 1: loss improved from inf to 0.52560, saving model to /home/utkarsh/Desktop/Sem-2/Deep Learning/2024PGCSDS14_Utkarsh Saxena_DeepLearning/DLProject/Caption.keras
18493/18493 ━━━━━━━━━━━━━━━━━━━━ 280s 15ms/step - loss: 0.6514
Epoch 2/20
18492/18493 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.3647
Epoch 2: loss improved from 0.52560 to 0.35455, saving model to /home/utkarsh/Desktop/Sem-2/Deep Learning/2024PGCSDS14_Utkarsh Saxena_DeepLearning/DLProject/Caption.keras
18493/18493 ━━━━━━━━━━━━━━━━━━━━ 282s 15ms/step - loss: 0.3647
Epoch 3/20
18492/18493 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.3209
Epoch 3: loss improved from 0.35455 to 0.31916, saving model to /home/utkarsh/Desktop/Sem-2/Deep Learning/2024PGCSDS14_Utkarsh Saxena_DeepLearning/DLProject/Caption.keras
18493/18493 ━━━━━━━━━━━━━━━━━━━━ 282s 15ms/step - loss: 0.3209
Epoch 4/20
18492/18493 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.3005
Epoch 4: loss 